In [3]:
from pathlib import Path
from llama_index import download_loader, VectorStoreIndex, load_index_from_storage, SummaryIndex
from llama_index.storage.storage_context import StorageContext

PDFReader = download_loader('PDFReader')
loader = PDFReader()

class PaperTitle:
    
    def __init__(self, name, about, file, key):
        self.name = name
        self.about = about
        self.file = file
        self.key = key
        
papers_titles = [
    PaperTitle('llama2', 'llama2 the name of large language models by metaAI', 'llama2.pdf', 'tools_for_llama2'),
    PaperTitle('LLM on Graph','Large Language Models using on Graph Structure','LLM_on_Graphs_A_Comprehensive_Survey.pdf', 'tools_for_GNN_or_recomendation_system')
]

paper_vector_index = {}
paper_summary_index = {}
for paper in papers_titles:
    try:
        storage_context = StorageContext.from_defaults(persist_dir=f"./storage/cache/{paper.key}_vector")
        paper_vector_index[paper.key] = load_index_from_storage(storage_context)
        
        storage_context = StorageContext.from_defaults(persist_dir=f"./storage/cache/{paper.key}_summary")
        paper_summary_index[paper.key] = load_index_from_storage(storage_context)
    except:
        documents = loader.load_data(file=Path(f"./assets/{paper.file}"))
        vector_index = VectorStoreIndex.from_documents(documents)
        paper_vector_index[paper.key] = vector_index
        vector_index.storage_context.persist(persist_dir=f"./storage/cache/{paper.key}_vector")

        paper_summary_index[paper.key] = SummaryIndex.from_documents(documents)
        paper_summary_index[paper.key].storage_context.persist(persist_dir=f"./storage/cache/{paper.key}_summary")

In [5]:
paper_summary_index # 각 paper에 대해서 vector index가 생성됨

{'tools_for_llama2': <llama_index.indices.list.base.SummaryIndex at 0x174f76c80>,
 'tools_for_GNN_or_recomendation_system': <llama_index.indices.list.base.SummaryIndex at 0x178a8a0b0>}

In [6]:
from llama_index.indices.postprocessor import KeywordNodePostprocessor
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.retrievers import VectorIndexRetriever
from llama_index.tools import QueryEngineTool, ToolMetadata

node_processor = KeywordNodePostprocessor(
    exclude_keywords=["commecial"]
)

agents = {}
query_engine_tools = []
paper_vector_engines = {}
paper_summary_engines = {}

for paper in papers_titles:
    retriever = VectorIndexRetriever(
        index = paper_vector_index[paper.key],
        similarity_top_k = 3,
    )
    paper_vector_engines[paper.key] = RetrieverQueryEngine(
        retriever = retriever,
        node_postprocessors = [node_processor]
    )
    
    paper_summary_engines[paper.key] = paper_summary_index[paper.key].as_query_engine()
    
    new_tool = QueryEngineTool(
        query_engine=paper_vector_engines[paper.key],
        metadata=ToolMetadata(
            name=f"{paper.key}_vector_tool",
            description=f"Useful for retrieving specific context from a paper {paper.name}. "
            f"Use When you need information related to {paper.about}"
            )
    )
    
    query_engine_tools.append(new_tool)
    
    new_tool = QueryEngineTool(
        query_engine=paper_summary_engines[paper.key],
        metadata=ToolMetadata(
            name=f"{paper.key}_summaey_tool",
            description=f"Useful for summary specific context from a paper {paper.name}. "
            f"Use When you need overview information related to {paper.about}"
            )
    )
    
    query_engine_tools.append(new_tool)
    
    from llama_index.agent import OpenAIAgent
    from llama_index.llms import OpenAI

    agent = OpenAIAgent.from_tools(query_engine_tools, llm=OpenAI(temperature=0, model='gpt-3.5-turbo-0613'), verbose=True)
    agents[paper.key] = agent

In [7]:
agents

{'tools_for_llama2': <llama_index.agent.openai.base.OpenAIAgent at 0x17568c430>,
 'tools_for_GNN_or_recomendation_system': <llama_index.agent.openai.base.OpenAIAgent at 0x17568ca90>}

In [9]:
from llama_index.schema import IndexNode

nodes = []
for paper in papers_titles:
    paper_summary = (
        f"This content contains papers title : {paper.name}. "
        f"Use thes index if you need to lookup specific information about {paper.about}."
    )
    node = IndexNode(text = paper_summary, index_id = paper.key)
    nodes.append(node)

In [10]:
vector_index = VectorStoreIndex(nodes)
vector_retriever = vector_index.as_retriever(simialrity_top_k=1)

In [13]:
from llama_index.retrievers import RecursiveRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.response_synthesizers import get_response_synthesizer

recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict = {'vector' : vector_retriever},
    query_engine_dict = agents,
    verbose=True
)

response_synthesizer = get_response_synthesizer(
    response_mode = 'compact',
)

query_engine = RetrieverQueryEngine.from_args(
    retriever = recursive_retriever,
    response_synthesizer = response_synthesizer
)

In [15]:
query_engine.query('Tell me about llama2')

Retrieving with query id None: Tell me about llama2
Retrieved node with id, entering: tools_for_llama2
Retrieving with query id tools_for_llama2: Tell me about llama2
Added user message to memory: Tell me about llama2
=== Calling Function ===
Calling function: tools_for_llama2_summaey_tool with args: {
  "input": "llama2"
}
Got output: Llama 2 is a set of large language models (LLMs) that are specifically designed for dialogue applications. These models, known as Llama 2-Chat, have undergone safety improvements and are considered a viable alternative to proprietary models. They come in different sizes, ranging from 7 billion to 70 billion parameters. The language generation capabilities of LLMs have been extensively studied, and there are concerns about bias, toxicity, privacy breaches, and potential malicious uses. It is crucial to address these safety challenges and develop strategies to mitigate harm, ensuring the responsible and positive use of Llama 2 models.

Got response: Llama 

Response(response='Llama 2 is a language model developed by Meta AI. It is available in different parameter sizes, including 7B, 13B, and 70B, and has both pretrained and fine-tuned variations. Llama 2 is based on an optimized transformer architecture and is trained using supervised fine-tuning and reinforcement learning with human feedback to align with human preferences for helpfulness and safety. It is primarily intended for commercial and research use in English, with tuned models suitable for assistant-like chat and pretrained models adaptable for various natural language generation tasks. It is important to note that Llama 2 should not be used in any way that violates applicable laws or regulations, in languages other than English, or in any way prohibited by the Acceptable Use Policy and Licensing Agreement for Llama 2. Developers who plan to use Llama 2 should perform safety testing and tuning tailored to their specific applications before deploying any applications of the mode

In [17]:
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI

agent = OpenAIAgent.from_tools(query_engine_tools, llm=OpenAI(temperature=0, model='gpt-3.5-turbo-0613'), verbose=True)

In [14]:
response = agent.chat('Tell me about llama2')
print(response.response)

Added user message to memory: Tell me about llama2
=== Calling Function ===
Calling function: tools_for_llama2_summaey_tool with args: {
  "input": "llama2"
}


KeyboardInterrupt: 

LLAMA 2 is a collection of pretrained and fine-tuned large language models (LLMs) developed by metaAI. Here are some key features of LLAMA 2:

1. Scale: LLAMA 2 consists of LLMs ranging in scale from 7 billion to 70 billion parameters. The larger models have more capacity to capture complex language patterns and generate high-quality text.

2. Fine-tuned for Dialogue: LLAMA 2 includes fine-tuned LLMs called LLAMA 2-Chat, which are specifically optimized for dialogue-based use cases. These models have been trained to generate contextually relevant and coherent responses in conversations.

3. Competitiveness: LLAMA 2-Chat models have demonstrated competitiveness with existing open-source chat models and are comparable to some proprietary models on evaluation sets. This means that LLAMA 2 can provide high-quality dialogue generation.

4. Optimized Transformer Architecture: LLAMA 2 utilizes an optimized transformer architecture, which is a state-of-the-art neural network architecture for n